In [2]:
import pandas as pd
import numpy as np
from collections import defaultdict

In [3]:
df = pd.read_csv("paragrapghs_with_topisc.csv.zip", index_col=0)
df.head()

/tmp/ipykernel_207448/3613625376.py:1: DtypeWarning: Columns (5,56,58,60,62,200) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("paragrapghs_with_topisc.csv.zip", index_col=0)


,Unnamed: 0,rsspp_index,index,company,year,sector,report_type,paragraph,original_text,cleaned_text,...,Корпоративное управление_importance,Корпоративное управление_base_words_count,Устойчивое развитие_percent_of_words,Устойчивое развитие_importance,Устойчивое развитие_base_words_count,!Кибербезопасность_percent_of_words,!Кибербезопасность_importance,!Кибербезопасность_base_words_count,cleaned_texttopics,count_words
0,0,151.0,151.0,reports/Щекиноазот,2023,"Химическая, нефтехимическая, парфюмерная",ЭО,0.0,"20.11.2023, 14:34",20.11.2023 14:34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,151.0,151.0,reports/Щекиноазот,2023,"Химическая, нефтехимическая, парфюмерная",ЭО,1.0,устойчивое развитие - щекиноазот,устойчивый развитие - щекиноазот,...,0.0,0.0,0.250000,1.0,1.0,0.000000,0.0,0.0,устойчивый развитие - щекиноазот,NaN
2,2,151.0,151.0,reports/Щекиноазот,2023,"Химическая, нефтехимическая, парфюмерная",ЭО,2.0,ru,ru,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,151.0,151.0,reports/Щекиноазот,2023,"Химическая, нефтехимическая, парфюмерная",ЭО,3.0,en,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,151.0,151.0,reports/Щекиноазот,2023,"Химическая, нефтехимическая, парфюмерная",ЭО,4.0,"устойчивое развитие esg – это environmental, s...",устойчивый развитие esg это environmental soci...,...,0.0,6.0,0.045872,8.0,10.0,0.009174,0.0,2.0,устойчивый развитие esg это environmental soci...,NaN


In [4]:
df.loc[df["company"] == "reports/ПАО «ВЫМПЕЛКОМ» ", ["company"]] = "reports/ПАО «ВЫМПЕЛКОМ»"

In [5]:
not_nan = df[df["max_topic_exp"].notna()]
# число абзацев соотвествующих теме
reports = not_nan.groupby(["company", "index", "max_topic_exp"])["year"].count()
reports.head()

company        index  max_topic_exp           
reports/1.1.-  110.0  Биоразнообразие             3
                      Вода                        6
                      Закупки и антикоррупция     3
                      Климат                      1
                      Корпоративное управление    2
Name: year, dtype: int64

# TF-IDF

## TF

In [6]:
topics_per_report = not_nan.groupby("index")["year"].count()
topics_per_report.head()

index
0.0    3325
1.0    2195
2.0    1515
4.0    3400
5.0    3065
Name: year, dtype: int64

In [7]:
tf = defaultdict(dict)
for (file_name, index, topic), row in reports.items():
    tf[index][topic] = row / topics_per_report[index]
tf = pd.DataFrame(tf).T

## IDF

In [8]:
idf = {}
for topic in df["max_topic_exp"].unique():
    if pd.isna(topic):
        continue
    idf[topic] = np.log(df["index"].nunique() / tf[topic].notna().sum())
idf = pd.Series(idf)

In [9]:
tf_idf = defaultdict(dict)
for idx, row in tf.iterrows():
    for topic in row.index:
        tf_idf[idx][topic] = row[topic] * idf[topic]

In [10]:
tf_idf = pd.DataFrame(tf_idf).T
tf_idf = tf_idf.fillna(0)
tf_idf.head()

,Биоразнообразие,Вода,Закупки и антикоррупция,Климат,Корпоративное управление,Обучение и развитие,Оплата труда,Отношения с инвесторами,Поставщики в целом,Потребители. Качество и безопас,...,Потребители. Здоровье и благопо,Потребители. Ценовая политика,Сотрудники. Наем и увольнение,Этика,Потребители. Персональные данны,Устойчивое развитие,Риски,Экологичность продукта,Малый и локальный бизнес,Поставщики. Экология
110.0,0.006054,0.011059,0.006054,0.001757,0.007374,0.007703,0.010531,0.020105,0.001588,0.009446,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
73.0,0.004174,0.001334,0.005009,0.003089,0.002669,0.012064,0.008090,0.039783,0.024137,0.006936,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
90.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
140.0,0.001361,0.003070,0.003682,0.001463,0.002632,0.007246,0.005251,0.055128,0.014043,0.003971,...,0.001593,0.002765,0.000572,0.000579,0.000000,0.0,0.0,0.0,0.0,0.0
100.0,0.002113,0.003584,0.002717,0.002891,0.004412,0.028479,0.005851,0.028470,0.002612,0.008478,...,0.000000,0.001159,0.000000,0.000000,0.009293,0.0,0.0,0.0,0.0,0.0


In [11]:
tf_idf.to_csv("tf_idf.csv")

In [12]:
raex_scores = pd.read_csv("raex_esg_with_rspp.csv", index_col=0)
raex_scores.head()

,№,Название,Код MOEX,Подотрасль,ESG-рейтинг,E Rank,E-рейтинг,S Rank,S-рейтинг,G Rank,...,Год последней оцененной отчетности,E_transformed,S_transformed,G_transformed,id,компания,сектор,год,тип отчета,ссылка на отчет
0.0,1,НЛМК,NLMK,Чёрная металлургия,AA,2,AA,2,AA,21,...,2021.0,2.0,2.0,3.0,714.0,ПАО «НЛМК»,Металлургическая и горнодобывающая,2021.0,ИО,/download/de45f8ddf53aa9ba80614d1c93e296fc/
1.0,2,«Полюс»,PLZL,Драгоценные металлы,AA,1,AAA,14,A,27,...,2021.0,1.0,3.0,3.0,887.0,ПАО «Полюс»,Металлургическая и горнодобывающая,2021.0,ОУР,/download/a79b0f48c2f54dc885ddf583ef4c402d/
2.0,3,«Уралкалий»,-,Агрохимикаты,A,6,BBB,1,AA,6,...,2021.0,4.0,2.0,2.0,1275.0,ПАО «Уралкалий»,"Химическая, нефтехимическая, парфюмерная",2021.0,ОУР,/download/0e88d52331c02d9939be212b2b95e562/
3.0,4,«ЭЛ5-Энерго»,ELFV,Электроэнергетика,A,9,BBB,9,A,2,...,2021.0,4.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
4.0,5,«Полиметалл»,POLY,Драгоценные металлы,A,4,A,5,A,12,...,2021.0,3.0,3.0,2.0,875.0,ПАО «Полиметалл»,Металлургическая и горнодобывающая,2021.0,ИО,/download/dd60bd2b772498057452e9b81e540671/


In [13]:
tf_idf

,Биоразнообразие,Вода,Закупки и антикоррупция,Климат,Корпоративное управление,Обучение и развитие,Оплата труда,Отношения с инвесторами,Поставщики в целом,Потребители. Качество и безопас,...,Потребители. Здоровье и благопо,Потребители. Ценовая политика,Сотрудники. Наем и увольнение,Этика,Потребители. Персональные данны,Устойчивое развитие,Риски,Экологичность продукта,Малый и локальный бизнес,Поставщики. Экология
110.0,0.006054,0.011059,0.006054,0.001757,0.007374,0.007703,0.010531,0.020105,0.001588,0.009446,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0
73.0,0.004174,0.001334,0.005009,0.003089,0.002669,0.012064,0.008090,0.039783,0.024137,0.006936,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0
90.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0
140.0,0.001361,0.003070,0.003682,0.001463,0.002632,0.007246,0.005251,0.055128,0.014043,0.003971,...,0.001593,0.002765,0.000572,0.000579,0.000000,0.0,0.0,0.000000,0.0,0.0
100.0,0.002113,0.003584,0.002717,0.002891,0.004412,0.028479,0.005851,0.028470,0.002612,0.008478,...,0.000000,0.001159,0.000000,0.000000,0.009293,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94.0,0.000000,0.021944,0.000000,0.020920,0.000000,0.013103,0.000000,0.005319,0.000000,0.007497,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.136249,0.0,0.0
151.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.014712,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0
144.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.008221,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0
15.0,0.004095,0.003378,0.006077,0.004141,0.001931,0.014697,0.007386,0.026849,0.012784,0.005998,...,0.000000,0.001014,0.000472,0.001912,0.002033,0.0,0.0,0.000000,0.0,0.0


In [14]:
assert len(tf_idf.index[~tf_idf.index.isin(raex_scores.index)]) == 0, (
    tf_idf.index[~tf_idf.index.isin(raex_scores["компания"])],
    raex_scores[~raex_scores["компания"].isin(tf_idf.index)]["компания"].unique(),
)

In [15]:
combined = tf_idf.join(raex_scores[["E_transformed", "S_transformed", "G_transformed"]])

In [16]:
combined.to_csv("tfidf_with_esg.csv")

In [17]:
topic_words = pd.read_csv("../topic_words.csv", index_col=0)
topic_words.head()

,topic,weight,type,word
meta,,,,
env,Экология в целом,5.0,0.0,экологичный
env,Экология в целом,5.0,0.0,природоохранный
env,Экология в целом,1.0,0.0,экология
env,Экология в целом,5.0,0.0,воздействие
env,Экология в целом,5.0,0.0,окружающая среда


In [18]:
gr = topic_words.groupby("meta")["topic"].apply(set)

In [19]:
topics = set([col.split("_")[0] for col in combined.columns if col.find("_") != -1])
topics = [topic for topic in topics if len(topic) > 1]

In [20]:
for (
    let,
    row,
) in gr.items():
    print(let, len(row))

env 9
gov 10
social 26


In [21]:
letter = {"env": ["E_transformed"], "social": ["S_transformed"], "gov": ["G_transformed"]}
for meta, col_set in gr.items():
    for col in col_set:
        found = False
        for c_col in combined.columns:
            if col in c_col:
                letter[meta].append(c_col)
                found = True
        # if not found:
        #     print(col)
for key, arr in letter.items():
    print(key, len(arr))

env 10
social 23
gov 6


# Fisrt

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score


def logreg_(data: pd.DataFrame, cur_letter: str):
    model = LogisticRegression()
    y_col = f"{cur_letter}_transformed"
    X = data.drop([y_col], axis=1)
    y = data[y_col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"{X_train.shape=} {X_test.shape=}")

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print("accuracy_score", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))

/home/samoed/.pyenv/versions/3.10.4/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [23]:
def map_to_ranges(value):
    if value <= 3:
        return 1
    elif value <= 6:
        return 2
    else:
        return 3

In [24]:
for cur_letter in ["E", "S", "G"]:
    col = f"{cur_letter}_transformed"
    combined[col] = combined[col].apply(map_to_ranges)

In [25]:
combined.head()

,Биоразнообразие,Вода,Закупки и антикоррупция,Климат,Корпоративное управление,Обучение и развитие,Оплата труда,Отношения с инвесторами,Поставщики в целом,Потребители. Качество и безопас,...,Этика,Потребители. Персональные данны,Устойчивое развитие,Риски,Экологичность продукта,Малый и локальный бизнес,Поставщики. Экология,E_transformed,S_transformed,G_transformed
0.0,0.002544,0.007603,0.005781,0.008120,0.001408,0.015174,0.006608,0.021194,0.014190,0.009272,...,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.0,1,1,1
1.0,0.001868,0.005012,0.005488,0.006811,0.000853,0.024514,0.004439,0.005816,0.016351,0.003006,...,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.0,1,1,1
2.0,0.001015,0.011742,0.007782,0.020179,0.002164,0.018358,0.003909,0.006629,0.007853,0.006018,...,0.0,0.0,0.00000,0.000000,0.0,0.001949,0.0,2,1,1
4.0,0.002261,0.005645,0.002488,0.008401,0.002066,0.010647,0.013486,0.038600,0.007413,0.005504,...,0.0,0.0,0.00000,0.000749,0.0,0.000000,0.0,1,1,1
5.0,0.002341,0.007866,0.008195,0.006552,0.003819,0.017145,0.008664,0.017439,0.008158,0.011624,...,0.0,0.0,0.00105,0.002494,0.0,0.000000,0.0,2,1,1


In [26]:
for cur_letter, letter_type in [("E", "env"), ("S", "social"), ("G", "gov")]:
    print(cur_letter, letter_type)
    logreg_(combined[letter[letter_type]], cur_letter)

E env
X_train.shape=(90, 9) X_test.shape=(23, 9)
accuracy_score 0.6086956521739131
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         7
           3       0.61      1.00      0.76        14

    accuracy                           0.61        23
   macro avg       0.20      0.33      0.25        23
weighted avg       0.37      0.61      0.46        23

S social
X_train.shape=(90, 22) X_test.shape=(23, 22)
accuracy_score 0.43478260869565216
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         4
           2       0.00      0.00      0.00         9
           3       0.43      1.00      0.61        10

    accuracy                           0.43        23
   macro avg       0.14      0.33      0.20        23
weighted avg       0.19      0.43      0.26        23

G gov
X_train.shape=(90, 5) X_test.shape=(23, 5)
accuracy_score 0.

/home/samoed/.pyenv/versions/3.10.4/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/samoed/.pyenv/versions/3.10.4/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/samoed/.pyenv/versions/3.10.4/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

In [27]:
combined["E_transformed"].value_counts()

3    77
2    32
1     4
Name: E_transformed, dtype: int64

# Second

In [28]:
from pymcdm.methods import TOPSIS

In [29]:
gr = (
    df.drop(
        columns=[
            "Unnamed: 0",
            "rsspp_index",
            "sector",
            "year",
            "report_type",
            "paragraph",
            "original_text",
            "cleaned_text",
            "max_topic_cos",
            "max_score_cos",
            "count_words",
            "second_max_topic_cos",
            "second_max_score_cos",
            "max_topic_exp",
            "max_score_exp",
            "second_max_topic_exp",
            "second_max_score_exp",
        ]
    )
    .groupby(["company", "index"])
    .mean()
    .dropna(how="all")
)
gr.head()

/tmp/ipykernel_207448/1102755827.py:24: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  .mean()


,,Экология в целом,Климат,Энергия,Воздух,Вода,Отходы и циклическая экономика,Биоразнообразие,Рекультивация земель,Экологичность продукта,!Персонал в целом,...,Этика_base_words_count,Корпоративное управление_percent_of_words,Корпоративное управление_importance,Корпоративное управление_base_words_count,Устойчивое развитие_percent_of_words,Устойчивое развитие_importance,Устойчивое развитие_base_words_count,!Кибербезопасность_percent_of_words,!Кибербезопасность_importance,!Кибербезопасность_base_words_count
company,index,,,,,,,,,,,,,,,,,,,,,
reports/1.1.-,110.0,0.001821,0.002850,0.008317,0.002750,0.008126,0.003903,0.004642,0.003788,0.005799,0.0,...,0.113281,0.019921,0.460938,0.886719,0.039843,0.167969,1.527344,0.001834,0.0,0.089844
reports/2 SCF Annual Report 2020 RUS,73.0,0.000398,0.003640,0.003335,0.002752,0.002794,0.002710,0.005188,0.001518,0.008647,0.0,...,0.066273,0.023735,0.292783,0.488879,0.100666,0.146618,0.634589,0.002016,0.0,0.046300
reports/2020 Annual Report РСБ,90.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.007780,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000367,0.000000,0.009174,0.000000,0.0,0.000000
reports/2021,140.0,0.001449,0.000834,0.003763,0.001557,0.005198,0.003144,0.003192,0.002004,0.005446,0.0,...,0.047106,0.046266,0.173253,0.358084,0.091810,0.089421,0.502196,0.001468,0.0,0.026148
reports/20210325ctpz1godovoi-otcet-za-2020-god,100.0,0.000899,0.000494,0.002799,0.001582,0.003923,0.004237,0.001727,0.001949,0.005364,0.0,...,0.065780,0.032802,0.180546,0.408677,0.088017,0.191043,0.624913,0.003020,0.0,0.054584


In [30]:
topics = topic_words.groupby("meta")["topic"].apply(set)

In [31]:
gr.reset_index(inplace=True)
gr = gr.set_index("index")
gr.head()

,company,Экология в целом,Климат,Энергия,Воздух,Вода,Отходы и циклическая экономика,Биоразнообразие,Рекультивация земель,Экологичность продукта,...,Этика_base_words_count,Корпоративное управление_percent_of_words,Корпоративное управление_importance,Корпоративное управление_base_words_count,Устойчивое развитие_percent_of_words,Устойчивое развитие_importance,Устойчивое развитие_base_words_count,!Кибербезопасность_percent_of_words,!Кибербезопасность_importance,!Кибербезопасность_base_words_count
index,,,,,,,,,,,,,,,,,,,,,
110.0,reports/1.1.-,0.001821,0.002850,0.008317,0.002750,0.008126,0.003903,0.004642,0.003788,0.005799,...,0.113281,0.019921,0.460938,0.886719,0.039843,0.167969,1.527344,0.001834,0.0,0.089844
73.0,reports/2 SCF Annual Report 2020 RUS,0.000398,0.003640,0.003335,0.002752,0.002794,0.002710,0.005188,0.001518,0.008647,...,0.066273,0.023735,0.292783,0.488879,0.100666,0.146618,0.634589,0.002016,0.0,0.046300
90.0,reports/2020 Annual Report РСБ,0.000000,0.000000,0.000000,0.000000,0.000000,0.007780,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000367,0.000000,0.009174,0.000000,0.0,0.000000
140.0,reports/2021,0.001449,0.000834,0.003763,0.001557,0.005198,0.003144,0.003192,0.002004,0.005446,...,0.047106,0.046266,0.173253,0.358084,0.091810,0.089421,0.502196,0.001468,0.0,0.026148
100.0,reports/20210325ctpz1godovoi-otcet-za-2020-god,0.000899,0.000494,0.002799,0.001582,0.003923,0.004237,0.001727,0.001949,0.005364,...,0.065780,0.032802,0.180546,0.408677,0.088017,0.191043,0.624913,0.003020,0.0,0.054584


In [32]:
def second_workflow(data: pd.DataFrame):
    matrix = data.to_numpy()
    weights = np.array([1 / matrix.shape[0]] * matrix.shape[1])
    types = np.ones(matrix.shape[1], dtype=int)
    body = TOPSIS()
    predictions = [round(preference, 2) for preference in body(matrix, weights, types)]
    return predictions

In [33]:
gr = gr.join(raex_scores["Название"])

In [35]:
gr["company"].unique()

array(['reports/ПАО «НЛМК»', 'reports/ПАО «Полюс»',
       'reports/ПАО «Уралкалий»', 'reports/ПАО «Полиметалл»',
       'reports/ПАО «ФосАгро»', 'reports/ПАО «ГМК «Норильский никель»',
       'reports/ПАО «Северсталь»', 'reports/ПАО «ЛУКОЙЛ»',
       'reports/ПАО «МТС»', 'reports/БФ «Система»',
       'reports/ПАО «Татнефть»', 'reports/Яндекс',
       'reports/Московская Биржа', 'reports/Юнипро', 'reports/ОК РУСАЛ',
       'reports/ПАО «НОВАТЭК»', 'reports/ПАО «НК «Роснефть»',
       'reports/АО «СУЭК»', 'reports/АО «ОХК Уралхим»',
       'reports/ПАО «РусГидро»', 'reports/АК «АЛРОСА»', 'reports/Газпром',
       'reports/ПАО Россети Московский регион', 'reports/Группа ЛСР',
       'reports/X5 Retail Group', 'reports/En+ Group ',
       'reports/ПАО «АЭРОФЛОТ»', 'reports/ПАО «Ростелеком»',
       'reports/АО «МХК «ЕвроХим»', 'reports/Магнит',
       'reports/ПАО «ФСК ЕЭС» («Россети») ', 'reports/«МЕТАЛЛОИНВЕСТ»',
       'reports/ОАО «РЖД»', 'reports/ПАО «ТМК»',
       'reports/Госкорпо

In [36]:
bad = ["Кибербезопасность", "Инновации", "Персонал в целом"]
gr["company"] = gr["company"].str.split("/").str[1]
for key, cur_topics in topics.items():
    tmp_topics = []
    for t in cur_topics:
        if t not in bad:
            tmp_topics.append(t)
        else:
            tmp_topics.append("!" + t)
    cur_topics = tmp_topics
    res = second_workflow(gr[cur_topics])
    pd.Series(res).set_axis(gr.index).to_csv(f"{key}.csv")
    gr[key] = res

In [37]:
bad = ["Кибербезопасность", "Инновации", "Персонал в целом"]

tmp_topics = []
for t in [v for _, val in topics.items() for v in val]:
    if t not in bad:
        tmp_topics.append(t)
    else:
        tmp_topics.append("!" + t)
cur_topics = tmp_topics
res = second_workflow(gr[cur_topics])
res = pd.Series(res).set_axis(gr.index)
gr["all_letters"] = res

In [38]:
gr["on_letters"] = second_workflow(gr[["env", "gov", "social"]])
gr.to_csv("raex_results.csv")

In [39]:
df = pd.read_csv("raex_results.csv", index_col=0)

In [40]:
df.tail()

,company,Экология в целом,Климат,Энергия,Воздух,Вода,Отходы и циклическая экономика,Биоразнообразие,Рекультивация земель,Экологичность продукта,...,Устойчивое развитие_base_words_count,!Кибербезопасность_percent_of_words,!Кибербезопасность_importance,!Кибербезопасность_base_words_count,Название,env,gov,social,all_letters,on_letters
152.0,Godovoi,0.00000,0.000000,0.003781,0.000000,0.001447,0.003106,0.001346,0.008763,0.008415,...,0.985348,0.001838,0.0,0.047619,Кордиант,0.06,0.40,0.16,0.22,0.50
154.0,«Промышленно-металлургический холдинг» (ПМХ) П...,0.00000,0.007795,0.000000,0.000000,0.000000,0.010577,0.000000,0.010250,0.000000,...,8.864865,0.003043,0.0,0.756757,«Промышленно-металлургический холдинг» (ПМХ),0.08,0.31,0.24,0.23,0.51
156.0,god,0.00000,0.001904,0.002744,0.001691,0.003844,0.000000,0.004862,0.000000,0.005536,...,1.028369,0.000313,0.0,0.056738,«Минудобрения»,0.04,0.08,0.04,0.05,0.13
157.0,Michelin,0.00000,0.010371,0.000000,0.000000,0.000000,0.009272,0.000000,0.000000,0.000000,...,0.025333,0.000000,0.0,0.000000,«Мишлен»,0.06,0.00,0.00,0.03,0.08
158.0,Петропавловск группа компаний Отчёт об устойчи...,0.00733,0.014552,0.005347,0.007796,0.014137,0.007382,0.006916,0.007734,0.007111,...,0.924004,0.006542,0.0,0.131603,"«Петропавловск», группа компаний",0.12,0.11,0.19,0.17,0.36
